In [43]:
!pip install nevergrad
!pip install yellowbrick

import sqlite3 as db
import pandas as pd
import json
import time
import statistics as stat
import random

import matplotlib.pyplot as plt
import seaborn as sns
import nevergrad as ng

from copy import deepcopy
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import BaggingRegressor

import yellowbrick
from yellowbrick.regressor import prediction_error

<center><h2><b>Leer DB</b></h2></center>

Primero de todo leemos la base de datos en un dataframe. Hemos limpiado cada tabla individualmente y las hemos juntado con LEFT JOIN para que no se pierda<br>
ninguna información de la tabla principal que es pacient.

In [44]:
def sql_query(q):
    conn = db.connect('../db/sqlite/eicu_v2_0_1_clean.sqlite3')
    df = pd.read_sql_query(q, conn)
    conn.close()
    
    return df

In [101]:
X_query = """
    SELECT age, hospitalid, admissionheight, hospitaladmitoffset, hospitaldischargeoffset, unitvisitnumber, admissionweight, unitdischargeoffset,
           avg_unit_stay, avg_hospital_stay, admission_bmi, acutephysiologyscore, apachescore, predictedicumortality,
           predictediculos, actualiculos, predictedhospitalmortality, predictedhospitallos, actualhospitallos, preopmi, preopcardiaccath,
           ptcawithin24h, unabridgedunitlos, unabridgedhosplos, actualventdays, predventdays, unabridgedactualventdays
    FROM patient P LEFT JOIN (SELECT *
                              FROM ApachePatientResult
                              GROUP BY patientunitstayid) A
    ON P.patientunitstayid = A.patientunitstayid
"""

X = sql_query(X_query)
y = X['unitdischargeoffset']

for col in X.columns.values:
    X[col] = X[col].fillna(-1)

In [102]:
X

,age,hospitalid,admissionheight,hospitaladmitoffset,hospitaldischargeoffset,unitvisitnumber,admissionweight,unitdischargeoffset,avg_unit_stay,avg_hospital_stay,admission_bmi,acutephysiologyscore,apachescore,predictedicumortality,predictediculos,actualiculos,predictedhospitalmortality,predictedhospitallos,actualhospitallos,preopmi,preopcardiaccath,ptcawithin24h,unabridgedunitlos,unabridgedhosplos,actualventdays,predventdays,unabridgedactualventdays
0,87,59,157.5,-2258,366,2,67.6,344,0,0,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.0000,-1.0,-1.0,-1.0,-1.0000,-1.0000,-1.0,-1.000000,-1.0
1,87,59,157.5,-8,2616,1,46.5,2250,344,366,3.387097,23.0,47.0,0.008247,0.722231,1.5625,0.037320,2.881710,1.8222,0.0,0.0,0.0,1.5625,1.8222,-1.0,-1.000000,-1.0
2,76,68,167.0,-1,1218,1,77.5,793,0,0,2.154839,43.0,60.0,0.015707,3.021671,0.5506,0.028932,6.032627,0.8465,0.0,0.0,0.0,0.5506,0.8465,-1.0,-1.000000,-1.0
3,34,56,172.7,-23,1138,1,60.3,1121,0,0,2.864013,25.0,25.0,0.001834,0.592446,0.7784,0.004253,2.196294,0.8063,0.0,0.0,0.0,0.7784,0.8063,-1.0,-1.000000,-1.0
4,61,68,177.8,-10,5263,1,91.7,1369,0,0,1.938931,26.0,37.0,0.009514,3.188109,0.9506,0.021407,10.930641,3.6618,0.0,0.0,0.0,0.9506,3.6618,-1.0,-1.000000,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515,62,459,165.1,-68242,36894,1,134.5,5394,0,0,1.227509,87.0,98.0,0.139750,7.764718,3.7458,0.407495,53.809676,50.0000,0.0,0.0,0.0,3.7458,73.0111,2.0,4.187875,2.0
2516,41,458,177.8,-1512,18482,2,127.0,4261,0,0,1.400000,46.0,46.0,0.009288,2.559068,2.9590,-1.000000,-1.000000,13.8847,1.0,1.0,0.0,2.9590,13.8847,2.0,1.384220,2.0
2517,41,458,177.8,-136,19858,1,127.0,1369,4261,18482,1.400000,24.0,24.0,-1.000000,-1.000000,0.9506,0.021816,6.984501,13.8847,0.0,0.0,1.0,0.9506,13.8847,1.0,2.080439,1.0
2518,72,458,177.8,0,5579,1,68.3,4166,0,0,2.603221,-1.0,-1.0,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.0000,-1.0,-1.0,-1.0,-1.0000,-1.0000,-1.0,-1.000000,-1.0


<center><h2><b>Transformación de columnas</b></h2></center>

In [ ]:
categs = {}

for col in ['gender', 'ethnicity', 'hospitalid', 'hospitaladmitsource']:
    categs[col] = list(set(X[col]))

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=234750)

No hemos puesto demasiada antención a la eliminación de outliers y normalización de columnas numéricas ya que nuestro principal regresor ha sido RandomForest. Hemos usado<br>
OneHot para las variables categóricas con un número limitado de columnas y directamente passthrough para las variables numéricas.

In [105]:
transformers = []

# Numerar columnas para el tranformador
for i in range(len(transformers)):
    transformers[i][2].append(i)

# Transformar la matriz
X_T_train = X_train #ColumnTransformer(transformers=transformers).fit_transform(X_train)
X_T_test  = X_test #ColumnTransformer(transformers=transformers).fit_transform(X_test)

# Mostrar el cambio en columnas
display('Train', print(X.shape, '->', X_T_train.shape))
display('Test ', print(X_test.shape, '->', X_T_test.shape))

(2520, 27) -> (2268, 27)


'Train'

None

(252, 27) -> (252, 27)


'Test '

None

<center><h2><b>Busqueda de Hiperparametros</b></h2></center>

Hemos usado Nevergrad para buscar los hiperparámetros de RandomForest y XBoost mediante un algorítmo genético, ya que son una cantidad importante de hiperparámetros.<br>

In [77]:
# Número aleatorio para la búsqueda de hiperparámetros
rand_n = random.randint(0, 100000000000)

# Funcion auxiliar para devolver un conjunto de scores y su media y desviación estandar
def cv_avg_std(reg, X, y, scoring):
    maes = cross_val_score(reg, X, y, cv=5, scoring=scoring)
    avg = stat.mean(maes)
    std_dev = stat.variance(maes)**(1/2)
    
    return maes, avg, std_dev

# Función de optimización para nevergrad
def optimize_spectral(n_estimators, min_samples_leaf, min_samples_split, max_depth, max_features, warm_start, bootstrap):
    reg = RandomForestRegressor(n_estimators=n_estimators, min_samples_leaf=min_samples_leaf,
                              min_samples_split=min_samples_split, max_depth=max_depth, random_state=rand_n,
                              max_features=max_features, bootstrap=bootstrap)

    maes, avg, std_dev = cv_avg_std(reg, X_T_train, y_train, 'neg_mean_absolute_error')
    _, r2, __ = cv_avg_std(reg, X_T, y, 'r2')
  
    print('AVG_MAE', avg, 'R2', r2, 'std_dev', std_dev, "[", n_estimators, ",", min_samples_leaf, ",", min_samples_split, ",", max_depth, ',', max_features, ',', warm_start, ',', bootstrap, '] - ', rand_n)

    return float('inf') if std_dev > 75 else -r2

instru = ng.p.Instrumentation(
    ng.p.Choice([x for x in range(40, 200)]), # n_estimators
    ng.p.Choice([x for x in range(1, 4)]), # min_samples_leaf
    ng.p.Choice([x for x in range(2, 5)]), # min_samples_split
    ng.p.Choice([x for x in range(30, 200)]), # max_depth
    ng.p.Choice([x for x in range(10, 200)]), # max_features
    ng.p.Choice([True, False]), # warm_start
    ng.p.Choice([True, False]) # bootstrap
)

# Descomentar para ejecutar

#optimizer = ng.optimizers.CM(parametrization=instru, budget=1000)
#recommendation = optimizer.minimize(optimize_spectral)
#print(recommendation.value)  # recommended value

<center><h2><b>Reducción dimensional</b></h2></center>

Hemos probado SVD para reducir el número de features, acelerar el tiempo de entreno y reducir el ruido que se haya podido originar<br>
por parte de features que no sean demasiados útiles, ya que nuestro conjunto de training es una matriz esparsa y no se puede usar PCA,<br>
sin embargo, no ha ayudado a mejorar el modelo, incrementando en su vez todas las medidas de error.

In [ ]:
def feature_n_for_explained_variance_ratio(svd, ratio):
    total = 0
    
    for i, n in enumerate(svd.explained_variance_ratio_):
        total += n
        
        if total >= ratio:
            return i+1
    
    return -1


from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=22)
_X_T_train = svd.fit_transform(X_T_train)

for n in (0.5, 0.75, 0.9, 0.99, 0.999, 0.9999):
    print("Numero de features para explicar un " + str(n*100) + "% de la varianza:",
          feature_n_for_explained_variance_ratio(svd, n))

#X_T = svd.transform(X_T)

<center><h2><b>Entrenamiento y calcular Error</b></h2></center>

Para el cálculo de error hemos usado tres medidas. Por una parte tenemos el MAE para calcular la distancia media a la variable objetivo que es el<br>
número de minutos que tenemos que predecir que un paciente va a permanecer en la UCI. Aparte del MAE también hemos considerado importante la desviación<br>
estandar del mismo ya que se trata de un sistema donde se pueden perder vidas si se hacen predicciones poco precisas. Por último hemos usado R2 para<br>
calcular la cantidad de varianza que explica el modelo y evitar overfitting.

In [97]:
# avg df: 3484.0752 = 2.420139 dias.
def cv_avg_std(reg, X, y, scoring):
    maes = cross_val_score(reg, X, y, cv=2, scoring=scoring)
    avg = stat.mean(maes)
    std_dev = stat.variance(maes)**(1/2)
    
    return maes, avg, std_dev

def make_df(datos_reg):
    error_df = pd.DataFrame()

    error_df['Regresor']               = datos_reg.keys()
    error_df['Average MAE']            = [ abs(dato['avg']) for dato in datos_reg.values() ]
    error_df['Standard Deviation MAE'] = [ dato['std_dev'] for dato in datos_reg.values() ]
    error_df['Average R2']             = [ dato['avg_r2'] for dato in datos_reg.values() ]
    error_df['time']                   = [ dato['time'] for dato in datos_reg.values() ]
    
    return error_df

In [106]:
# Linear regression 
lreg = LinearRegression()

rfreg = RandomForestRegressor(
        n_estimators = 121,
        min_samples_leaf = 2,
        min_samples_split = 3,
        max_depth = 91,
        max_features = 189,
        warm_start = False,
        bootstrap = False,
        random_state = 14684358)

xboostreg = XGBRegressor()

votingreg = VotingRegressor([('xboost', lreg), ('rfreg', rfreg)], weights=[1, 3])

datos_reg = {}
regressors = [
    ('Lineal', lreg),
    ('Random Forest', rfreg),
    ('XGBRegressor', xboostreg),
    ('VotingRegressor', votingreg),
]

# Medir tiempo y hacer predicciones para cada regresor
for reg_name, reg in regressors:
    start_time = time.time()

    maes, avg, std_dev = cv_avg_std(reg, X_T_train, y_train, 'neg_mean_absolute_error')
    maes, r2, _ = cv_avg_std(reg, X_T_train, y_train, 'r2')
    
    datos_reg[reg_name] = { 'avg': avg, 'std_dev': std_dev, 'time': time.time() - start_time }
    datos_reg[reg_name]['avg_r2'] = r2

df_reg = make_df(datos_reg)
df_reg

,Regresor,Average MAE,Standard Deviation MAE,Average R2,time
0,Lineal,2.571649e-12,3.654030e-13,1.000000,0.022842
1,Random Forest,7.360582e+01,2.083755e+01,0.984221,3.875248
2,XGBRegressor,6.185283e+01,2.880076e+00,0.995880,0.329715
3,VotingRegressor,5.520436e+01,1.562816e+01,0.991124,4.025902


In [ ]:
sns.set_palette("Paired")
fig, axs = plt.subplots(ncols=3, nrows=1,figsize=(20, 5))

sns.barplot(data=df_reg, x='Regresor', y='Average MAE', ax=axs[0])
sns.barplot(data=df_reg, x='Regresor', y='Standard Deviation MAE', ax=axs[1])
sns.barplot(data=df_reg, x='Regresor', y='Average R2', ax=axs[2])

Por último hacemos un plot con el error de cada modelo para comparar cuanto se distancian de un resultado ideal.

In [ ]:
rfreg.fit(X_T_train, y_train)
pred = rfreg.predict(X_T_test)

In [ ]:

#fig, axs = plt.subplots(ncols=4, nrows=1,figsize=(20, 10))

#diff = y_test - pred
#diff = [ x for x in diff if abs(x) < 100 ] # Eliminar 3/250 outliers que cambian la escala

#sns.regplot(x='unitdischargeoffset', y="age", data=X_test);
#sns.lineplot(data=[i for i in range(len(diff))], label="y_test", ax=ax)
#sns.lineplot(data=[diff[i] + i for i in range(len(diff))], label="pred", ax=ax)

#axs = axs.flatten()

#for row in ax:
#axs[0, 0]
err = prediction_error(lreg,       X_T_train, y_train, X_T_test, y_test)#, ax=axs[0])
err = prediction_error(rfreg,      X_T_train, y_train, X_T_test, y_test)#, ax=axs[1])
err = prediction_error(xboostreg,  X_T_train, y_train, X_T_test, y_test)#, ax=axs[2])
err = prediction_error(votingreg,  X_T_train, y_train, X_T_test, y_test)#, ax=axs[3])

Basandonos en estos plots podemos ver que todos se acercan bastante a la pendiente de 45º. LinearRegressor y GBRegressor se<br>
alejan un poco más, pero es natural ya que no hemos tenido tiempo de buscar unos hiperparámetros razonables o hacer eliminación<br>
de outliers. En el caso de RandomForest no le afectan tanto estos outliers y se hace posible una explicación más amplia de la varianza.

<center><h2><b>Conclusión</b></h2></center>

Tras evaluar todos estos modelos podemos observar que mientras que LinearRegressor tiene un ligeramente mayor R2 y un tiempo de ejecución<br>
envidiable, la diferencia en MAE no compensa usarlo. RandomForestRegressor se muestra como un contendiente fuerte, pero hemos podido mejorarlo<br>
haciendo un ensemble de Voting con el anterior. XBoost probablemente mejoraría dado el tiempo suficiente para hacer una búsqueda de hiperparámetros<br>
exhaustiva, pero no hemos tenido tiempo. A todo esto pensamos que el VotingRegressor muestra unos resultados aceptables.<br>

En un nivel más abstracto podemos ver que tiene una media de error de ~262 minutos ~ 4 horas y una desviación estandar de alrededor de media hora. Creemos<br>
que es importante contar con una desviación estandar pequeña para poder predecir la estancia de los pacientes con cierta fiabilidad y teniendo en cuenta<br>
que el tiempo medio que un paciente está en la UCI en E.E.U.U es de alrededor de 10 días y que la media del dataset es de 2.4 días, creemos que es un<br>
resultado interesante con el cual se podrían hacer, hipotéticamente, predicciones útiles.<br>

También es merecedor de mención que este resultado no supera a APACHE. Un modelo únicamente entrenado con la variable unbridgedunitlos que tiene un índice<br>
de correlación de más del 98% con la variable que estamos buscando tiene un R2 de un 98.5% y un MAE y desviación estandar menores. No hemos querido mejorar<br>
el modelo hasta esos niveles tanto por falta de tiempo como por miedo de overfitting, aunque Random Forest es notorio por su tendencia a evitar overfitting.<br>

En general y para terminar podemos decir que ha sido un reto interesante donde nuestros principales problemas han sido la interpretación de 30 tablas de datos<br>
(muchas para la experiencia que habíamos tenido hasta ahora) y la interpretación de dichos datos, sobretodo los datos vacíos (donde algunos han acabado teniendo<br>
sentido) como el significado de dichos datos en un contexto médico.